In [1]:
import json
import nltk
import re
import string
from nltk.corpus import stopwords 
import demoji
from nltk.chunk import conlltags2tree, tree2conlltags
import spacy
stop_words = set(stopwords.words('english')) 
demoji.download_codes()

... OK (Got response in 0.22 seconds)
Writing emoji data to C:\Users\victo\.demoji/codes.json ...
... OK


In [2]:
with open('gg2020.json', encoding='utf8') as json_file:
    data = [json.loads(line) for line in json_file]

In [3]:
data = json.load(open('gg2013.json'))

In [4]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [5]:
def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [20]:
clean_data = []
count = 0
for tweet in data:
    tweet_text = tweet['text']
    no_http = re.sub('http://\S+|https://\S+', '', tweet_text)
    remove_tag = strip_all_entities(no_http)
    #emoji_list = demoji.findall(remove_tag)
    #tweet_token = nltk.word_tokenize(remove_tag)
    #filtered_tweet = tweet_token
    #filtered_tweet = [w for w in filtered_tweet if not w in emoji_list]
    #filtered_tweet = [w for w in filtered_tweet if not w in string.punctuation]
    #filtered_tweet = [w for w in filtered_tweet if w.isalnum()]
    #filtered_tweet = [w.lower() for w in filtered_tweet]
    clean_data.append(no_http)

In [21]:
import itertools
clean_data.sort()
unique_clean_data = list(set(clean_data))

In [22]:
print(len(unique_clean_data))
print(len(clean_data))


127315
174643


In [23]:
num_k = []
for elem in clean_data:
    if elem not in num_k:
        num_k.append(elem)
print(len(num_k))


127315


#### Find hosts

In [9]:
# search host
host_list = ['host', 'hosts', 'hosting', 'hosted']
host_tweet = []
for tweet in clean_data:
    for test_word in host_list:
        if test_word in tweet:
            host_tweet.append(tweet)
            break

In [10]:
tag = nltk.pos_tag(clean_data[0])
pattern = 'NP: {<DT>?<JJ>*<NN>}'
cp = nltk.RegexpParser(pattern)
cs = cp.parse(tag)
iob = tree2conlltags(cs)

In [11]:
i = 0
nlp = spacy.load("en_core_web_sm")
doc = nlp(' '.join(word for word in clean_data[i]))
print(' '.join(word for word in clean_data[i]))
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [16]:
name_list = []
for tweet in host_tweet:
    doc = nlp(tweet)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name_list.append(ent.text)

In [17]:
unique_list = list(set(name_list))
counter = []
for item in unique_list:
    counter.append(name_list.count(item))
sort_counter = sorted(range(len(counter)), key=lambda k: counter[k], reverse=True)    

In [18]:
possible_list = []
for i in range(5):
    name = unique_list[sort_counter[i]]
    possible_list.append(name)
reduced_possible_list = []
for i in range(5):
    test = possible_list[i]
    conflict = 0
    for j in range(5):
        if test in possible_list[j] and i != j:
            conflict = 1
            break
    if test not in reduced_possible_list and conflict == 0:
        reduced_possible_list.append(test)
# search first
hosts = []
hosts.append(reduced_possible_list[0])
host_idx = possible_list.index(hosts[0])
max_num = counter[sort_counter[host_idx]]
# search second
name_pair = []
for name1 in reduced_possible_list:
    for name2 in reduced_possible_list:
        count = 0
        for tweet in host_tweet:
            tweetstr = ' '.join(map(str, tweet)) 
            if name1 + ' and ' + name2 in tweetstr or name2 + ' and ' + name1 in tweetstr:
                count += 1
        if count > max_num * 0.1:
            name_pair.append([name1, name2, count])
name_pair.sort(key=lambda x: x[2], reverse=True)
if len(name_pair) > 0:
    hosts = [name_pair[0][0],name_pair[0][1]]

In [19]:
print(hosts)

['Will Ferrell']


#### Find awards

In [ ]:
awards_bag_of_words = ['-','performance', 'actress', 'actor', 'supporting', 'role', 'director', 'motion', 'picture', 'drama','animated', 'film', 'song','comedy', 'musical', 'language', 'foreign','screenplay', 'orginal', 'television', 'tv', 'series', 'mini-series', 'mini']

In [ ]:
pattern = re.compile(r'(?<=w[oi]n[s]\s)[Bb]est.*')


In [ ]:
tweet_sep = ['for','who',' s ','made',' | ', 'with' ,' at', ' http', ' #', '(','.', ',', '!', '?','\\', ':', ';', '"', "'",'the','but','although', 'made']

In [ ]:
for tweet in extracted_text_data:
    award = re.findall(pattern, tweet)
    if award:
        for a in award:
            a = a.lower()
            for sep in tweet_sep:
                if sep in a:
                    a = a.split(sep)[0]
            #if "/" in a:
            #    a = a.replace('/',' or ')
            awards.append(a)

In [108]:
awards_list = {}
for tweet in clean_data:
    if "best" in tweet and "performance" in tweet and "award" in tweet:
        collect = []
        for word in tweet:
            if word == "best" and len(collect) == 0:
                collect.append(word)
            elif word == "performance" and len(collect) == 1:
                collect.append(word)  
            elif word == "by" and len(collect) == 2:
                collect.append(word)                 
            elif word == "award" and len(collect) > 3:
                collect.append(word)
                collect_str = ' '.join(map(str, collect))
                if collect_str not in awards_list:
                    awards_list[collect_str] = 0
                awards_list[collect_str] += 1
                collect = []
            elif len(collect) >= 3:
                collect.append(word)                

In [109]:
print(awards_list)

{'best performance by an actor in a motion picture drama last night s ceremony was the first major award': 1, 'best performance by an actor in a motion picture musical or comedy by golden globe award': 1, 'best performance by an actor in a supporting role in any motion picture by golden globe award': 1, 'best performance by an actor award': 1, 'best performance by an actor in a motion picture drama by golden globe award': 1, 'best performance by an actress in a supporting role award': 1, 'best performance by an actor in a limited series or motion picture made for television award': 1, 'best performance by an actor in a supporting role in a motion picture this is pacino s 18th nomination with 4 wins and the cecil b demille award': 1, 'best performance by an actor in a limited series because it s jharrel s award': 1, 'best performance by an actor in a motion picture drama upon accepting his award': 1}


In [110]:
awards_list = {}
for tweet in clean_data:
    if "the" in tweet and "award" in tweet:
        #print(tweet)
        collect = []
        for word in list(reversed(tweet)):  
            if word == "award":
                collect.insert(0,word)
            elif word == "the" and len(collect) > 0:
                #print(collect_str)
                collect_str = ' '.join(map(str, collect))
                if len(collect) > 2 and len(collect) <= 4 and 'golden' not in collect_str:
                    doc = nlp(collect_str)
                    for ent in doc.ents:
                        if ent.label_ == "PERSON":                    
                            if collect_str not in awards_list:
                                awards_list[collect_str] = 0
                            awards_list[collect_str] += 1
                            break
                        else:
                            break

                collect = []
            elif len(collect) > 0:
                collect.insert(0,word)

In [90]:
print(awards_list)

{'brightest genius award': 1, 'carol burnett award': 109, 'cecille b demille award': 1, 'carroll burnett award': 2, 'prestigious carol burnett award': 1, 'chairman s award': 1, 'carol burnett show award': 1, 'green goddess award': 1, 'event carol burnett award': 1, 'galton an award': 1, 'honorary carol burnett award': 1, 'carol burnet award': 1, 'inaugural carol burnett award': 1, 'jussie smollett award': 1, 'director s guild award': 1, 'official kickoff to award': 1}


In [ ]:
for tweet in clean_data:
    tweetstr = ' '.join(map(str, tweet)) 